In [ ]:
import numpy as np
from scipy.interpolate import griddata
import matplotlib.pyplot as plt
from matplotlib.collections import LineCollection
import cmocean
%matplotlib notebook

In [ ]:
# Increase or reduce the interpolation resolution with these numbers.
nx = 101
ny = 101

In [ ]:
with open("./Coastlines/coastlines_rot.xyz", 'r') as fi:
    lines = fi.readlines()
    segment_list = []
    segment = []
    for line in lines[1:-1]:
#         print('line=',line.rstrip())
        if line.startswith('>'):
            segment_list.append( segment )
#             print(segment, line_segments)
            segment = []
#             print(segment)
            continue
        sxy = line.split()
        x, y = float(sxy[0]), float(sxy[1])
#         print(sxy, x, y)
        segment.append( [x,y] )
#         print( 'segment=', segment )
    segs = np.array( segment_list, dtype=object )
#     print(line_arrays)

In [ ]:
def interpolate_data(data):
    x = data[:,0]-500.0
    y = data[:,1]-200.0
    z = data[:,2]
    print(x.shape, y.shape, z.shape, np.vstack((x,y)).transpose().shape)
    xi = np.linspace(0.0, 4000.0, nx)
    yi = np.linspace(0.0, 3600.0, ny)
    Xi, Yi = np.meshgrid(xi, yi)
    zi = griddata( np.vstack((x,y)).transpose(), z, (Xi, Yi), method='linear')
    
    return xi, yi, zi


In [ ]:
data = np.loadtxt("./Topo/Topo.xyz")
Txi, Tyi, Tzi = interpolate_data( data )
# plotSurf( xi, yi, zi, "Topography", line_segments)

with open("./Topo/Topo_rect.xyz","w") as fo:
    for j in range(ny):
        for i in range(nx):
            # Save interface elevations in km 
            # so that all the coordinates can be scaled to meters 
            # consistently in Trelis.
            print("{0:g} {1:g} {2:g}".format(Txi[i], Tyi[j], Tzi[j,i]*1e-3), file=fo)

In [ ]:
data = np.loadtxt("./Moho/Moho.xyz")
Mxi, Myi, Mzi = interpolate_data( data )
# plotSurf( xi, yi, zi, "Moho Depth", line_segments )

with open("./Moho/Moho_rect.xyz","w") as fo:
    for j in range(ny):
        for i in range(nx):
            # Save interface elevations in km 
            # so that all the coordinates can be scaled to meters 
            # consistently in Trelis.
            print("{0:g} {1:g} {2:g}".format( Mxi[i], Myi[j], Mzi[j,i]*1e-3), file=fo)

In [ ]:
data = np.loadtxt("./LAB/LAB.xyz")
Lxi, Lyi, Lzi = interpolate_data( data )
# plotSurf( xi, yi, zi, "LAB Depth", line_segments )


with open("./LAB/LAB_rect.xyz","w") as fo:
    for j in range(ny):
        for i in range(nx):
            # Save interface elevations in km 
            # so that all the coordinates can be scaled to meters 
            # consistently in Trelis.
            print("{0:g} {1:g} {2:g}".format( Lxi[i], Lyi[j], Lzi[j,i]*1e-3), file=fo)

In [ ]:
fig, ax = plt.subplots(2,2,figsize=(9,6))
fig.tight_layout(pad=2)
#     ax.contour(xi, yi, zi, cmap="inferno")
# Topo
line_segments1 = LineCollection(segs, linewidths=2,
                               colors='white', linestyle='solid')
pcm = ax[0,0].pcolor(Txi, Tyi, Tzi*1e-3, cmap=cmocean.cm.topo, vmin=-8, vmax=8) #"inferno")
fig.colorbar(pcm, ax=ax[0,0], label="Topography (km)")
# ax[0,0].add_collection(line_segments1)
ax[0,0].set_aspect( 1 )
# ax[0,0].set_title( "Topography" )
ax[0,0].set_xlabel("x (km)")
ax[0,0].set_ylabel("y (km)")

# Moho
line_segments2 = LineCollection(segs, linewidths=1,
                               colors='white', linestyle='solid')
pcm = ax[0,1].pcolor(Mxi, Myi, Mzi*1e-3, cmap=cmocean.cm.thermal, vmin=-60, vmax=0)
fig.colorbar(pcm, ax=ax[0,1], label="Moho Depth (km)")
ax[0,1].add_collection(line_segments2)
ax[0,1].set_aspect( 1 )
# ax[0,1].set_title( "Topography" )
ax[0,1].set_xlabel("x (km)")
ax[0,1].set_ylabel("y (km)")

# LAB
line_segments3 = LineCollection(segs, linewidths=1,
                               colors='white', linestyle='solid')
pcm = ax[1,0].pcolor(Lxi, Lyi, Lzi*1e-3, cmap=cmocean.cm.solar, vmin=-300, vmax=0)
fig.colorbar(pcm, ax=ax[1,0], label="LAB (km)")
ax[1,0].add_collection(line_segments3)
ax[1,0].set_aspect( 1 )
# ax[1,0].set_title( "Topography" )
ax[1,0].set_xlabel("x (km)")
ax[1,0].set_ylabel("y (km)")

ax[1,1].axis('off')

plt.savefig("boundaries.png")